<a href="https://colab.research.google.com/github/jaquelline/Alguns_Trabalhos/blob/main/Analise_Sobrevida_Sksurv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scikit-survival

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.0/49.0 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 30.3 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.3.2
    Uninstalling scikit-learn-1.3.2:
      Successfully uninstalled scikit-learn-1.3.2


In [ ]:
from google.colab import drive
from pathlib import Path
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from sksurv.util import Surv
from sksurv.linear_model import CoxnetSurvivalAnalysis
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sksurv.metrics import concordance_index_censored

# Carregar os dados
file_path = '/content/drive/My Drive/Notebooks_Prontuarios/processed_modelo_survival.csv'
data = pd.read_csv(file_path)

# Preparar os dados no formato necessário para sksurv
data["event"] = data["recidiva"].astype(bool)
data["duration"] = data["duracao"]

# Selecionar as variáveis relevantes
relevant_columns = [
    'idade_diagnostico', 'her2_valor', 'receptor_progesterona_valor',
    'receptor_estrogenio_valor', 'ki67_valor', 'invasao_linfonodos_valor',
    'linfonodo_axilar_comprometido_valor', 'largura_tumor', 'altura_tumor',
    'profundidade_tumor', 'grau_tumor_valor_1.0', 'grau_tumor_valor_2.0',
    'grau_tumor_valor_3.0', 'grau_tumor_valor_4.0'
]

X = data[relevant_columns]
y = Surv.from_dataframe("event", "duration", data)

# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Ajustar o modelo de Cox com regularização Lasso
estimator = make_pipeline(
    StandardScaler(),
    CoxnetSurvivalAnalysis(l1_ratio=1.0, alphas=[0.2])
)
estimator.fit(X_train, y_train)

# Prever no conjunto de teste
y_pred = estimator.predict(X_test)

# Calcular o C-index no conjunto de teste
c_index = concordance_index_censored(y_test['event'], y_test['duration'], y_pred)

print("C-index: {:.3f}".format(c_index[0]))

C-index: 0.583


In [ ]:
!pip freeze > requirements.txt

In [ ]:
from google.colab import files
files.download('requirements.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
model_file_name = "/content/drive/My Drive/Notebooks_Prontuarios/onco_survival_model_sksurv.pkl"

# Save model using Booster  - Problem with predict ('numpy.ndarray' object has no attribute 'feature_names')
#bt._Booster.save_model(model_file_name)

# Save model using sklearn XGBClassifier API - Problem with predict ('XGBClassifier' object has no attribute '_le')
#bt.save_model(model_file_name)


# Save model using pickle - Recommended according to https://github.com/dmlc/xgboost/pull/3829
# Use joblib which is considered better than pickling
import joblib
joblib.dump(estimator, model_file_name)